<참고>
- https://pytorch.org/docs/stable/generated/torch.load.html
- https://pytorch.org/docs/stable/generated/torch.save.html#torch.save
- https://pytorch.org/docs/stable/notes/serialization.html#saving-loading-tensors

#### torch.load(f, map_location=None, pickle_module=pickle, *, weights_only=False, mmap=None, **pickle_load_args)

- torch.save()로 부터 저장된 객체를 로드한다.
- torch.load()는 파이썬의 UnPickling 기능을 사용하며, 텐서의 기반이 되는 저장소를 특별하게 처리한다.
- `먼저 CPU에서 역직렬화(DeSerialized) 한 다음, 저장된 장치로 이동한다`.
- 만약 이것이 실패하면(예를 들어, 런타임에 특정 장치가 없어서) 예외가 발생. 그러나 map_location 인수를 사용하여 대체 장치에 동적으로 매핑할 수 있다.
  
- map_location이 호출되는 경우, 두개의 인수(storage 및 location)을 사용하여 직렬화된 각 저장된 저장소에 대해 한번 호출된다.
- storage: storage 인수는 cpu에 있는 초기 역 직렬화 이다.
- map_location: 직렬화된 각 저장소에는 저장된 장치를 식별하는 위치 테그가 연결되어 있으며, 이 테그는 map_location에 전달되는 두번째 인수이다.  
`직렬화된 저장소(location)에 저장된 것을 map_location이 cpu(default)(storage)로 역 직렬화 하여 가지고 온다.`
  
- 빌트인 location 테스는 CPU 텐서의 경우 'cpu', CUDA 텐서의 경우 'cuda:device_id' 이다. 
- map_location은 None 혹은 저장소를 반환해야 한다. 
- map_location이 스토리지를 반환하는 경우 이는 이미 올바른 장치로 이동된 최종 역직렬화된 개체로 사용된다(??)
- 그렇지 않으면 torch.load()는 마치 map_location이 지정되지 않은 것 처럼 기본 동작으로 돌아 간다(?)
  
- 만약 map_location이 torch.device 객체이거나, 장치 테크를 포함하는 문자열인 경우, 모든 텐서가 로드되어야 하는 위치를 나타낸다.
- 그렇지 않고, map_location이 딕셔너리인 경우, 파일(키)에 나타나는 위치 테그를 저장소(값)을 넣을 위치를 지정하는 테그로 다시 매핑하는데 사용된다.
- 사용자 확장은 torch.serialization.register_package()를 사용하여 자체 위치 테그와 테그 지정 및 역직렬화 방법을 등록할 수 있다.

In [ ]:
>>> torch.load('tensors.pt')
# Load all tensors onto the CPU
>>> torch.load('tensors.pt', map_location=torch.device('cpu'))
# Load all tensors onto the CPU, using a function
>>> torch.load('tensors.pt', map_location=lambda storage, loc: storage)

# Load all tensors onto GPU 1
>>> torch.load('tensors.pt', map_location=lambda storage, loc: storage.cuda(1))

# Map tensors from GPU 1 to GPU 0
>>> torch.load('tensors.pt', map_location={'cuda:1': 'cuda:0'})

# Load tensor from io.BytesIO object
>>> with open('tensor.pt', 'rb') as f:
...     buffer = io.BytesIO(f.read())

>>> torch.load(buffer)

# Load a module with 'ascii' encoding for unpickling
>>> torch.load('module.pt', encoding='ascii')